In [ ]:
!pip install bltk
!pip install nltk
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%matplotlib inline
import bltk
import nltk
import numpy as np
import regex as re
import pandas as pd
nltk.download('punkt')
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC, SVC
from nltk.tokenize import word_tokenize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from bltk.langtools import Tokenizer, remove_stopwords
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LinearRegression, SGDClassifier, LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer

In [ ]:
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/thesis_dataset.xlsx')
 
data = df
X = data['Review']
X.dropna(inplace=True)
print(X)
 
y = data['Label']
y.dropna(inplace=True)
len(y)
 
lebel=[]
for item in y:
    lebel.append(int(item))
#print(lebel)

In [ ]:
pattern=r"([!@#$%^&*(),.?-_+\`·:={}|<>A-Za-z0-90.0।Ⓒ©®»/০-৯'])"
def preprocess(string):
  result=re.sub(pattern," ",str(string))
  vlue=word_tokenize(result)
  #stopword_removed=remove_stopwords(vlue,level='hard')
  Single_list=list(dict.fromkeys(vlue))
  return Single_list
new=[]
for lines in X:
  result = preprocess(lines)
  new.append(result)
print(new)

In [ ]:
negations = ["না", "নি", "নাই", "নাহ"]

In [ ]:
def sentiments(text):
 
  sentiwords = pd.read_excel('/content/drive/My Drive/Thesis_Final_Works/sentiword (1).xlsx')
  positive_words = sentiwords['Positive'].tolist()
  negative_words = sentiwords['Negative'].tolist()
  positive_count = 0
  negetive_count = 0
  neutral_count = 0
  contradictory = 0
 
  length = len(text)
 
  for first_word in text:
    if first_word in positive_words:
      positive_count += 1
    elif first_word in negative_words:
      negetive_count += 1
    else:
      neutral_count += 1
 
  # for j in range(0, length-2):
  #   if text[j] in contra_postive and text[j+1] in negative_words:
  #     #if text[j+1] in negations:
  #       #contradictory +=1
  #     if text[j+2] in negations:
  #       contradictory +=1
  for j in range(0, length-2):
    if text[j] in negative_words:
      if text[j+1] in negations:
        contradictory +=1
      elif text[j+2] in negations:
        contradictory +=1
  
 
  if contradictory > 0:
    return 3 
  elif negetive_count > 0:
    if positive_count == 0:
      return 1
    elif negetive_count >= positive_count:
      return 2
    else:
      return 3
  elif positive_count > 0:
    if negetive_count == 0:
      return 4
    elif positive_count  >  negetive_count:
      return 3
    else:
      return 2
  elif (positive_count > 0 ) and (negetive_count > 0) and (positive_count == negetive_count):
    return 2
  else:
    return 0
 
output_label = []
#print(new)
for sen in new: 
  ouput = sentiments(sen) 
  output_label.append(ouput)
print(output_label)

In [ ]:
print(output_label)
print(lebel)

In [ ]:
X_one=np.array(lebel)
X = np.reshape(X_one, (-1, 1))
 
y_one=np.array(output_label)
y1=np.reshape(y_one, (-1, 1))
X_train,X_test,y_train,y_test= train_test_split(X,y1,test_size=0.3, random_state=43)
print(len(X_train))
print(len(X_test))

In [ ]:
from sklearn import metrics
 
X_train=X_train.reshape(-1,1)
 
text_clf=Pipeline([('Classifier', LinearSVC())])
text_clf.fit(X_train,y_train)
 
text_clf1=Pipeline([('Classifier', RandomForestClassifier())])
text_clf1.fit(X_train,y_train)
 
prediction= text_clf.predict(X_test)
prediction1 = text_clf1.predict(X_test)
print(metrics.confusion_matrix(y_test,prediction))
print(classification_report(y_test,prediction))
print(metrics.accuracy_score(y_test,prediction))
 
print(f"\n\n{metrics.confusion_matrix(y_test,prediction1)}")
print(classification_report(y_test,prediction1))
print(f"\n\n\nAccuracy : {metrics.accuracy_score(y_test,prediction1):2.2%}\n\n\n\n")

In [ ]:
import seaborn as sns
text_clf1.fit(X_train, y_train)
y_predicted = text_clf1.predict(X_test)
cm = confusion_matrix(y_test, y_predicted)
sns.heatmap(cm, annot = True, fmt= 'd')

In [ ]:
CM=confusion_matrix(y_test,prediction1)
print(CM)
print(classification_report(y_test,prediction1))
#print(classification_report(y_test,prediction))
count=0
TP=0
FN=0
FP=0
TN=0
Sum=0
I=0
for i in range(0,len(CM)):
  for j in range(0,len(CM)):
    Sum=Sum+CM[i][j]
 
 
for i in range(0,len(CM)):
  TP=0
  FN=0
  FP=0
  TN=0
  for j in range(0,len(CM)):
    if i==j:
      TP=CM[i][j]
    else:
      FN=FN+CM[i][j]
      FP=FP+CM[j][i]
  TN=Sum-TP-FP-FN
  accuracy = (TP+TN) / (TP+FN+FP+TN)
  precision = TP / (TP+FP)
  recall = TP / (TP + FN)
 
  f1_score = (2*precision*recall) / (precision+recall)
  specificity = TN / (TN+FP)
  sensitivity = recall
  print(f"\n\nTP:{TP} FN:{FN} FP:{FP} TN:{TN} SUM:{Sum}")
  print('Precision: {:2.2%}'.format(precision))
  print('Recall: {:2.2%}'.format(recall))
  print('F1 Score: {:2.2%}'.format(f1_score))
  print('Specificity: {:2.2%}'.format(specificity))
  print('Sensitivity: {:2.2%}'.format(sensitivity))

In [ ]:
prediction_text=input()
prediction_text_clen=preprocess(prediction_text)
prediction_emotion=np.reshape(sentiments(prediction_text_clen),(1,-1))
prediction_output=text_clf1.predict(prediction_emotion)
if prediction_output == 4:
  print("\nREVIEW: Very Positive")
elif prediction_output == 3:
  print("\nREVIEW: Positive")
elif prediction_output == 1:
  print("\nREVIEW: Very Negative")
elif prediction_output == 2:
  print("\nREVIEW: Negative")
else:
  print("\nREVIEW: Neutral")
  print("\n\n")